<a href="https://colab.research.google.com/github/BenWilop/WSG_games/blob/main/playground_WSG_games.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import dotenv
import wandb
import os

dotenv.load_dotenv(os.path.join("/homes/55/bwilop/wsg/private/", "vscode-ssh.env"))
api_key = os.getenv("WANDB_API_KEY")
wandb.login(key=api_key)

data_folder = "/homes/55/bwilop/wsg/data/"
experiment_folder = "/homes/55/bwilop/wsg/experiments/"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: benwilop (benwilop-rwth-aachen-university). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /homes/55/bwilop/.netrc


In [2]:
%load_ext autoreload
%autoreload 2
import torch as t
from torch.nn.functional import cross_entropy, softmax

# from jaxtyping import Float
import matplotlib.pyplot as plt

from wsg_games.tictactoe.evals import *
from wsg_games.tictactoe.data import *
from wsg_games.tictactoe.game import *

from wsg_games.tictactoe.analysis.analyse_data import *
from wsg_games.tictactoe.analysis.visualize_game import *

from wsg_games.tictactoe.train.create_models import *
from wsg_games.tictactoe.train.save_load_models import *
from wsg_games.tictactoe.train.train import *
from wsg_games.tictactoe.train.finetune import *
from wsg_games.tictactoe.train.pretrain import *

DEVICE = t.device("cuda" if t.cuda.is_available() else "cpu")
print(DEVICE)

cuda


# Load Data & Models

In [16]:
project_name_pretrain = "tictactoe/tictactoe_pretraining"
project_name_finetune = "tictactoe/tictactoe_finetuning4"
weak_model_size = "small"
strong_model_size = "medium"
index = 9

# Load data
(
    tictactoe_train_data,
    tictactoe_weak_finetune_data,
    tictactoe_val_data,
    tictactoe_test_data,
) = load_split_data(data_folder + "tictactoe/", device=DEVICE, index=index)

# Load models
weak_model = load_model(
    project_name_pretrain,
    weak_model_size,
    Goal.WEAK_GOAL,
    experiment_folder,
    device=DEVICE,
    index=index,
)
strong_baseline_model = load_model(
    project_name_pretrain,
    strong_model_size,
    Goal.WEAK_GOAL,
    experiment_folder,
    device=DEVICE,
    index=index,
)
strong_model = load_model(
    project_name_pretrain,
    strong_model_size,
    Goal.STRONG_GOAL,
    experiment_folder,
    device=DEVICE,
    index=index,
)
finetuned_model = load_finetuned_model(
    project_name_finetune,
    weak_model_size,
    strong_model_size,
    experiment_folder,
    DEVICE,
    index,
)

# Print evaluations
(
    weak_loss,
    _,
) = quick_evaluation("weak_model", weak_model, tictactoe_test_data)
strong_baseline_loss, _ = quick_evaluation(
    "strong_baseline_model", strong_baseline_model, tictactoe_test_data
)
quick_evaluation("strong_model", strong_model, tictactoe_test_data)
weak_finetuned_loss, _ = quick_evaluation(
    "finetuned_model", finetuned_model, tictactoe_test_data
)
print(
    "Performance Gap Recovered (PGR): ",
    (weak_loss - weak_finetuned_loss) / (weak_loss - strong_baseline_loss),
)

Loading model from /homes/55/bwilop/wsg/experiments//tictactoe/tictactoe_pretraining/experiment_9_small_weak_2025-05-12-21-59_fvjbgvh3.pkl
Moving model to device:  cuda
Loading model from /homes/55/bwilop/wsg/experiments//tictactoe/tictactoe_pretraining/experiment_9_medium_weak_2025-05-12-21-59_a3tjenfa.pkl
Moving model to device:  cuda
Loading model from /homes/55/bwilop/wsg/experiments//tictactoe/tictactoe_pretraining/experiment_9_medium_strong_2025-05-12-22-13_901icw46.pkl
Moving model to device:  cuda
Moving model to device:  cuda
weak_model
weak_loss:  0.7512524127960205
strong_loss:  1.1474272012710571
strong_baseline_model
weak_loss:  0.5308648347854614
strong_loss:  1.312455177307129
strong_model
weak_loss:  1.508345365524292
strong_loss:  0.4748372733592987
finetuned_model
weak_loss:  0.7023875117301941
strong_loss:  0.8003960847854614
Performance Gap Recovered (PGR):  0.2217225739623366


In [19]:
from dictionary_learning.dictionary_learning import CrossCoder
from nnsight import LanguageModel

/homes/55/bwilop/wsg/WSG_games/dictionary_learning/dictionary_learning/utils.py:38: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  np.bool: th.bool,


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [18]:
!ls

dictionary_learning	     setup.cfg	 uv_venv    WSG_games.egg-info
playground_crosscoder.ipynb  setup.py	 uv_venv2   wsgvenv
playground_WSG_games.ipynb   test.ipynb  wandb	    wsgvenv4
README.md		     test.py	 wsg_games
